In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from transformers import Trainer
from torch.cuda.amp import autocast
import torch
import os
import pandas as pd

In [ ]:
data = pd.read_csv('/kaggle/input/flickr-image-dataset/flickr30k_images/results.csv',delimiter='|')

In [ ]:
from sklearn.model_selection import train_test_split
train_ds, test_ds = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
from transformers import AutoTokenizer, CLIPTextModel
from transformers import CLIPProcessor, CLIPModel


model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
model1 = CLIPVisionModel.from_pretrained("openai/clip-vit-base-patch32")
model2 = CLIPTextModel.from_pretrained("openai/clip-vit-base-patch32")
tokenizer = AutoTokenizer.from_pretrained("openai/clip-vit-base-patch32")

In [ ]:
class CLIPTrainer(Trainer):
    # computes loss w/o label smoothing
    def compute_loss(self, model, inputs, return_outputs=False):
        outputs = model(**inputs, return_loss=True)
        return outputs["loss"]

    def prediction_step(self, model, inputs, prediction_loss_only, ignore_keys):
        inputs = self._prepare_inputs(inputs)
        with torch.no_grad():
            if self.use_amp:
                with autocast():
                    loss = self.compute_loss(model, inputs)
            else:
                loss = self.compute_loss(model, inputs)
        return (loss, None, None)
    
trainer = CLIPTrainer(model, model.parameters(),
                          train_dataset=train_ds,
                          eval_dataset=test_ds,
                          )

trainer.train()

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

for epoch in range(num_epochs):
    for batch in your_data_loader:
        inputs = processor(batch["texts"], batch["images"], return_tensors="pt", padding=True, truncation=True)
        outputs = model(**inputs)
        loss = your_loss_function(outputs, batch["labels"])
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()